In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import random

In [ ]:
ratingsFilePath = 'ml-1m/ratings.dat'
itemFilePath = 'ml-1m/movies.dat'
trainingFilePath = 'data/training.csv'
testFilePath = 'data/test.csv'

ratingsFileSep = '::'
itemFileSep = '::'

In [ ]:
data_file = pd.read_table(ratingsFilePath, sep=ratingsFileSep, header=None, engine='python')
item_file = pd.read_table(itemFilePath, sep=itemFileSep, header=None, engine='python')

In [ ]:
users = np.unique(data_file[0])
items = np.unique(item_file[0])

numberOfUsers = len(users)
numberOfItems = len(items)

item_indices, user_indices = {}, {}

for i in range(len(items)):
    item_indices[items[i]] = i

for i in range(len(users)):
    user_indices[users[i]] = i

In [ ]:
V = sp.lil_matrix((numberOfUsers, numberOfItems))
T = sp.lil_matrix((numberOfUsers, numberOfItems))
for line in data_file.values:
    u, i, r, time = map(int, line)
    V[user_indices[u], item_indices[i]] = r
    T[user_indices[u], item_indices[i]] = time

In [ ]:
ratio = 0.3
training = []
test = []
for user in range(V.shape[0]):
    nonzero = np.nonzero(V[user, :])[1]
    testsize = len(nonzero) * ratio
    while len(nonzero) > testsize:
        x = random.randint(0, len(nonzero) - 1)
        training.append((user + 1, items[nonzero[x]], V[user, nonzero[x]], T[user, nonzero[x]]))
        nonzero = np.delete(nonzero, x)
    for item in nonzero:
        test.append((user + 1, items[item], V[user, item], T[user, item]))

In [ ]:
training_file = open(trainingFilePath, 'r+')
test_file = open(testFilePath, 'r+')
training_file.truncate()
test_file.truncate()
for row in training:
    training_file.write(str(row[0]) + "," + str(row[1]) + "," + str(int(row[2])) + "," + str(int(row[3])) + "\n")
training_file.close()
for row in test:
    test_file.write(str(row[0]) + "," + str(row[1]) + "," + str(int(row[2])) + "," + str(int(row[3])) + "\n")
test_file.close()